In [5]:
from numpy import dot
from numpy.linalg import norm
import openai
import copy
import random
random.seed(4)

In [6]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
def get_response(prompt=None, content=None):
    openai.api_type = "azure"
    openai.api_base = "https://bigo-chatgpt-9.openai.azure.com/"
    openai.api_version = "2023-03-15-preview"
    # key1: 19ea901e8e10475da1bb0537abf8e5a4
    # key2: 548e5c0c2aff453e932948927a27bde6
    openai.api_key = "19ea901e8e10475da1bb0537abf8e5a4"

    # role : system|user|assistant
    gpt_config = {'engine': 'gpt-35-turbo',
                'role': 'user',
                }
    message_list = [
            {"role": "system",
            "content": f"{content}"},
            {"role": "user", "content": f"{prompt}"}
            ]
    response = openai.ChatCompletion.create(
        engine=gpt_config['engine'],
        temperature=0.6,
        messages=message_list
    )
    return response['choices'][0]['message']['content']

## 定义迷宫

In [10]:
### 迷宫的边界矩阵和占有矩阵是用来在Web界面成相的，所以用矩阵表示。而表示一些活动、事件的是一个字典，那些活动事件也同时都是字典
### 这些都是一个现有的大型矩阵，存储在这个位置：.\generative_agents\environment\frontend_server\static_dirs\assets\the_ville\matrix\maze
### world\sector\arena大多是用来搞前端的展示的，所以这里不进行定义，game_object才有交互
maze_dict = {'world':int(),
            'sector':int(),
            'arena':int(),
            'game_object':int(),
            'events': list()}
maze = [[dict() for i in range(10)] for j in range(10)]

### 简单定义一些对应的场景
world_dic = {0:'森林',   
            1:'港口小镇',
            2:'乡村',
            3:'沙漠',
            4:'都市',
            5:'大海'}

In [11]:
### 定义一些场景，并为每个场景加入一个随机事件
event_prompt = ("请根据场景：{world}\n"
                "生成一个随机事件。用一段话描述或概括该随机事件。讲述事件的主要人物，经过，结果。只返回事件内容。")
for i in range(len(maze)):
    for j in range(len(maze[0])):
        maze[i][j]['world'] = random.randrange(0, 6)
        if 'events' in maze[i][j]:
            event = get_response(event_prompt.format_map({'world':world_dic[maze[i][j]['world']]}))
            maze[i][j]['events'].append(event)
        else:
            maze[i][j]['events'] = []
            event = get_response(event_prompt.format_map({'world':world_dic[maze[i][j]['world']]}))
            maze[i][j]['events'].append(event)

In [12]:
maze[1][2]['world'], maze[1][2]['events']

(2,
 ['事件内容：在乡村，一只失踪的家禽被找到了。\n\n经过：乡村的居民们发现，他们家的一只鸡突然失踪了。他们开始四处寻找，但是却没有找到。几天后，一位居民在他的花园里发现了一只相当瘦弱的鸡。此时，他们意识到这只鸡已经离家出走了几天。\n\n结果：居民们将鸡带回家，并给它喂食和喝水。几天后，这只鸡恢复了健康，并开始产蛋。居民们非常高兴，因为这只鸡对他们来说非常重要，他们非常感激那位发现鸡的居民。'])

## 定义人格

In [30]:
### 定义persona大类
class Persona():
    def __init__(self, name, persona, skills, interests, associative_memory=None):
        self.name = name
        self.persiona = persona
        self.skills = skills
        self.interests = interests
        self.asso_mem = associative_memory if associative_memory is not None else None

    def get_basic_info_prompt(self):
        prompt_template = (
            "你是沙盒世界的一员，你的名字是{name}。\n"
            "以下是你的性格:\n"
            "{persona}\n\n"
            "你掌握的技巧:\n"
            "{skills}\n\n"
            "你的兴趣爱好: {interests}\n"
        )
        return prompt_template.format_map({'name':self.name, 'persona':self.persiona, 'skills':self.skills, 'interests':self.interests})
    
    def get_summarized_asso_mem(self):
        if self.asso_mem is None or self.asso_mem is []:
            print("还没有历史记忆")
            return 
        else:
            events_prompt = ''
            for i in range(len(self.asso_mem)):
                event = self.asso_mem[i]
                events_prompt = events_prompt + f'事件: {event}'
            summarize_prompt = '请总结这些过往的事件:' + events_prompt + '每个事件用一句话去总结'
            #print(summarize_prompt)
            summarized_asso_mem = get_response(summarize_prompt)
            summarized_events = [eve for eve in summarized_asso_mem.split('\n') if eve != ""]
            return summarized_events

In [31]:
### 现在初始化几个人格
zhangsan_dic = {'name': "张三",
                'persona': "温和友善： 张三是一个亲切友好的人，总是愿意帮助他人，因此在社交圈中备受喜爱。\n勤奋和有责任感： 他非常注重工作，总是尽最大努力完成任务。他喜欢有组织和计划的生活，这使他在工作和生活中都表现出色。\n耐心： 张三拥有出色的耐心，不易冲动，能够冷静地应对压力和困难。\n好奇心旺盛： 他对世界充满好奇，喜欢学习新事物，经常探索新的兴趣领域。",
                'skills': "沟通技巧： 张三是一位出色的沟通者，擅长倾听他人并表达自己的观点，这使他在职业和社交场合都表现出色。\n烹饪： 他对烹饪非常有天赋，经常烹制美味的餐点，也喜欢尝试各种各样的烹饪风格。\n音乐才能： 张三会弹奏吉他，享受音乐的世界，也喜欢在朋友和家人聚会时演奏音乐。\n编程技能： 他拥有出色的计算机编程技能，是一位优秀的软件工程师。",
                'interests': "徒步旅行： 张三热爱大自然，经常进行徒步旅行，探索山川河流，欣赏自然之美。\n阅读： 他是一位书虫，热衷于各种类型的书籍，特别是科幻小说和历史书籍。\n篮球： 张三喜欢打篮球，经常和朋友一起在周末参加篮球比赛。"}
zhangsan = Persona(name=zhangsan_dic['name'], persona=zhangsan_dic['persona'], skills=zhangsan_dic['skills'] ,interests=zhangsan_dic['interests'])


lisi_dic= {'name': "李四",
                'persona': "自信和坚定： 李四有坚定的信念，自信而果断，能够在压力下保持冷静。\n创造力： 他是一位富有创造力的思考者，常常能够提出新颖的想法和解决问题的方法。\n友善和关心他人： 李四非常关心他人，善待朋友和家人，经常主动帮助别人，建立了强大的社交网络。\n冒险精神： 他喜欢尝试新事物，勇于冒险，经常参加各种刺激的户外活动。",
                'skills': "绘画和艺术： 李四是一位出色的艺术家，擅长绘画和创作艺术作品，他的作品经常展示在当地画廊。\n瑜伽： 他是一名有经验的瑜伽修行者，经常进行冥想和身体训练，保持身心健康。\n写作： 李四热爱写作，擅长创作诗歌和散文，也有一些出版的短篇小说。\n户外冒险： 他喜欢登山、潜水和徒步旅行，经常在自然中寻找刺激和冒险。",
                'interests': "探索古迹： 李四热爱历史和文化，经常前往古迹和博物馆，探索人类历史的奥秘。\n音乐： 他是一位音乐爱好者，喜欢演奏吉他，也热衷于参加音乐节和音乐会。\n烹饪： 李四对烹饪有浓厚兴趣，经常烹制各种美食，特别喜欢尝试国际美食。"}
lisi = Persona(name=lisi_dic['name'], persona=lisi_dic['persona'], skills=lisi_dic['skills'] ,interests=lisi_dic['interests'])
lisi.get_basic_info_prompt(), lisi.get_summarized_asso_mem()

还没有历史记忆


('你是沙盒世界的一员，你的名字是李四。\n以下是你的性格:\n自信和坚定： 李四有坚定的信念，自信而果断，能够在压力下保持冷静。\n创造力： 他是一位富有创造力的思考者，常常能够提出新颖的想法和解决问题的方法。\n友善和关心他人： 李四非常关心他人，善待朋友和家人，经常主动帮助别人，建立了强大的社交网络。\n冒险精神： 他喜欢尝试新事物，勇于冒险，经常参加各种刺激的户外活动。\n\n你掌握的技巧:\n绘画和艺术： 李四是一位出色的艺术家，擅长绘画和创作艺术作品，他的作品经常展示在当地画廊。\n瑜伽： 他是一名有经验的瑜伽修行者，经常进行冥想和身体训练，保持身心健康。\n写作： 李四热爱写作，擅长创作诗歌和散文，也有一些出版的短篇小说。\n户外冒险： 他喜欢登山、潜水和徒步旅行，经常在自然中寻找刺激和冒险。\n\n你的兴趣爱好: 探索古迹： 李四热爱历史和文化，经常前往古迹和博物馆，探索人类历史的奥秘。\n音乐： 他是一位音乐爱好者，喜欢演奏吉他，也热衷于参加音乐节和音乐会。\n烹饪： 李四对烹饪有浓厚兴趣，经常烹制各种美食，特别喜欢尝试国际美食。\n',
 None)

In [32]:
### 接收到当前附近发生的信息后，会搜索过去发生的一些事，这些事存在于associative memory里
### 这些事会以一个总结的形式展现出来
associative_memory = ["在一个繁忙的都市街头，一名街头艺术家正在创作一幅令人惊叹的涂鸦艺术作品，吸引了路人的目光。突然，一位年轻的音乐家拿起他的吉他，开始演奏一首感人的原创歌曲，创造出一场意外的街头表演，将人们聚集在一起，创造了一刻充满艺术和音乐的魔幻时刻。",
                      "在一个宁静的湖畔小屋，一位退休的科学家突然收到一封神秘的信件，信中提到了一个失踪多年的研究合作伙伴。信件中附带了一张旧照片，一个未解之谜的地点，以及一条神秘的信息。这突如其来的消息将引发一段旅程，揭示了多年来的谜团和未解之谜。",
                      "在一个古老的城市，一座已废弃多年的废墟突然传来神秘的音乐。当当地居民前往调查时，他们发现一个不可思议的场景：一支看似来自古老时代的音乐团队在废墟中演奏，音乐回荡在城市中，令人陶醉。这不寻常的音乐会引发了关于时间旅行和神秘力量的疑问，城市陷入了充满奇迹的事件之中。",
                      "在一个宁静的农村小镇，一颗明亮的陨石坠落在一片麦田中，引起了当地居民的轰动。当科学家和农夫们赶到现场进行研究时，他们惊讶地发现，这颗陨石散发出神秘的光芒，对植物生长和周围环境产生了神奇的影响。小镇陷入了一场关于科学、奇迹和自然力量的调查之中，引发了深刻的思考和探索。",
]
zhangsan.asso_mem = associative_memory
lisi.asso_mem = associative_memory
lisi.get_summarized_asso_mem()

['1. 街头艺术家和音乐家创造了一场充满艺术和音乐的魔幻时刻。',
 '2. 一封神秘信件引发了一段揭示多年未解之谜的旅程。',
 '3. 废墟中的古老音乐团队引发了关于时间旅行和神秘力量的疑问。',
 '4. 陨石坠落引发了小镇围绕科学、奇迹和自然力量的调查。']

# agents行为流程

## Perceive

In [49]:
### 在perceive过程中，agents先获取附近的场景信息
### loc是agents当前所处位置，attn_width是能见度
def get_nearby_events(maze, loc, attn_width):
    row, column = loc[0], loc[1]
    perceive = []
    for i in range(max(row-attn_width,0), min(9,row+attn_width)+1):
        for j in range(max(column-attn_width,0), min(9,column+attn_width)+1):
            if 'events' in maze[i][j]:
                for event in maze[i][j]['events']:
                    eve = f"场景：{world_dic[maze[i][j]['world']]}， 事件: " + event
                    perceive.append(eve)
    return perceive

In [50]:
perceived = get_nearby_events(maze, (5,5), 1)
perceived

['场景：都市， 事件: 一位名叫李明的年轻人在下班回家的路上，突然发现自己的手机不见了。他开始搜索自己的口袋和背包，但没有找到。他感到非常沮丧和困惑，因为他的手机里有很多重要的联系人和文件。他决定回到公司看看是否把手机落在了办公室里。在回去的路上，他发现自己的手机被一个小偷偷走了。他开始追逐小偷，但小偷跑得很快，很快就消失在人群中。李明非常失望，但他决定报警，并通过互联网远程锁定了自己的手机。他希望警方能够找到小偷并追回他的手机。',
 '场景：港口小镇， 事件: 在港口小镇，一艘货船突然发生漏水事故，船上的船员们开始惊慌失措。经过紧急处理，港口小镇的消防队和救援队赶到现场，展开了紧急救援行动。在数小时的努力下，漏水被成功控制，船员们也被成功救出。虽然货船受损严重，但是没有造成人员伤亡，港口小镇的救援行动也得到了大家的赞扬。',
 '场景：沙漠， 事件: 在沙漠中，一个旅行家遇到了一场突如其来的沙尘暴。他被迫停下来等待暴风雨过去，但是暴风雨越来越大，他的食物和水也快用完了。突然，他发现一个迷路的小女孩也被困在了沙漠中。他决定冒着风沙救出她，但是他们的视线被沙尘暴遮盖，很快他们就迷失了方向。在经过长时间的努力后，他们终于找到了一处避风的石洞，成功地躲过了沙尘暴。在石洞里，他们发现了一些水和食物，这让他们得以继续前行。最终，他们成功地走出了沙漠。',
 '场景：乡村， 事件: 某天，乡村的村民们发现一只受伤的小动物躺在路边，它的身上还有些许血迹。村民们都很关心这只小动物，但又不知道应该怎么办。最终，一个勇敢的小男孩主动提出要把小动物带回家进行救治。他和他的家人一起花了很多时间护理这只小动物，最终小动物康复了，也成为了小男孩的好朋友。这个事件展现了乡村人们的善良和勇气，也给小男孩带来了一份珍贵的友谊。',
 '场景：乡村， 事件: 在乡村里，一个老奶奶正在田间劳作，突然发现自己的眼前一片模糊，看不清周围的一切。她非常担心自己的眼睛出了问题，于是赶紧去找医生看病。经过医生的检查，发现老奶奶的眼睛只是由于长时间劳作疲劳造成的，需要休息一段时间。医生给老奶奶开了一些药物和眼罩，让她好好休息，保护眼睛。老奶奶听从医生的建议，休息了几天后，眼睛就恢复了。',
 '场景：森林， 事件: 在森林里，一只狐狸意外地发现了一个被困在陷阱里的小兔子。狐狸本能地想要抓住兔子，但同时也感到了同

## Retrieve

In [33]:
### 设定一个previous memory的库，对于这些previous memory，我们以总结的形式展现
### 代码里的previous memory有很多信息，memory本身，对memory思考thought，还有一些描述description，这里为了简单只用summarized memory
### previous memory都存在向量数据库，用的是向量数据库的cos的匹配算法搜索的，这里不好演示，所以只是让gpt对目前发生的事，自己去找一个最相关的然后筛选

previous_evns = ["在一个古老的图书馆，一位年轻的研究员翻阅一本陈旧的书时，意外发现了一张被藏在书页间的古老地图。这张地图指示着一处未知的迷失之地，充满传说和谜题。激发了他的好奇心，研究员决定展开一场探险，与一队多样化的探险家一同前往，寻找这个神秘之地，解开隐藏其中的古代宝藏和谜题。",
                "在一个宁静的山区村庄，突然爆发了一场罕见的气象现象，天空中出现了奇怪的光球，伴随着雷声和强烈的电磁波。居民们困惑不解，然后这些光球突然合成一个巨大的光柱，将村庄中心的一棵古老树包围，树上开始绽放出华丽的花朵。这一事件引发了全村的惊讶和研究，居民们开始寻求解释这一不可思议的奇迹。",
                "在一个繁忙的都市中，一座老旧的建筑被拆除，工人们在废墟下发现了一座长时间失落的地下城市。这个地下城市完全保存了古老的建筑、壁画和文物，引发了考古学家和历史学家的广泛关注。随着调查的深入，他们逐渐揭示了这座城市的历史和目的，以及它如何与现代世界相互联系，成为一个引人入胜的谜题。",
                "在一个宁静的海边小镇，一群年轻人在一次夜间沙滩露营活动中，意外发现了一个闪烁的海洋生物，它们发光的蓝色色彩在夜晚的海浪中熠熠生辉。这些生物被发现时引发了一场科学界的热烈兴趣，他们开始研究这些闪烁生物的生态和行为习性，而小镇居民也开始举办夜间海滩观赏活动，将这一自然奇观分享给更多人。这一不寻常的发现让小镇的夏季活动变得更加独特和引人入胜。",
                "在一个宁静的乡村，当地居民突然发现一群奇怪的植物在一夜之间迅速生长，这些植物散发出迷人的香气。随着时间的推移，居民们开始注意到这些植物有治愈属性，可以用于传统医学。这一不寻常的自然发现引发了一场全民研究和探索的热潮，以及有关草药学和自然疗法的新兴兴趣。当地居民和科学家开始合作，以挖掘这些植物的潜力，并寻求将其用于医学和保健领域。",
                "在一个遥远的沙漠地区，一支考古队伍在沙丘下发现了一座古老的城市废墟，这座城市被认为已经失落了数千年。当考古学家们进一步研究时，他们在废墟中发现了一座神秘的地下迷宫，里面藏有未解之谜、宝藏和精密的文物。这一发现引发了国际科研团队的兴趣，他们将共同努力解开这座城市和地下迷宫的谜团，揭示其丰富的历史和文化秘密。这一事件也将引发考古界和历史研究领域的突破性发展。",
                "在一个古老的寺庙，一名修道士突然宣布，他已悟道，能够实现心灵投影。他邀请附近的居民前来经验这一奇迹，许多人前来寻求智慧和启示。在修道士的引导下，人们开始经历心灵旅程，探索内心的迷宫，这一事件将深刻地影响寺庙和附近社区的生活，带来精神和哲学的变革。同时，学者和科学家也前来研究这一现象，试图理解其背后的科学和神秘力量。",
                "在一个偏远的山区小镇，当地居民突然开始报告关于神秘灯光和不明飞行物的目击事件。这些光点在夜空中移动，产生令人难以置信的色彩和图案。随着时间的推移，这些事件变得越来越频繁，吸引了全球的媒体和研究人员的关注。这一现象引发了广泛的科学研究和讨论，人们纷纷提出各种解释和理论，而小镇的居民也开始团结起来，创建了一个神秘灯光观测社区，以记录和研究这些令人惊奇的现象。这一事件将不仅改变小镇的命运，也将改变我们对宇宙的理解。",
                "在一个宁静的湖泊边，一群垂钓者和游泳者突然目击到湖中出现了一只巨大的水生生物，被形容为类似传说中的“水怪”。这一惊人的发现迅速引发了媒体的广泛报道和探险者的前来，试图捕捉这一生物的证据。科学家也前来研究，以探索这一未解之谜。湖泊周边的小镇开始迎接了游客的涌入，这一事件让小镇焕发了生气，并引发了对自然界奥秘的浓厚兴趣。",
                "在一个繁忙的都市公园，一名街头魔术师正在表演惊人的魔术表演，吸引了围观的人们。然而，当他试图让一只鸽子从他的鎏金帽子中飞出时，却突然消失了，让人目瞪口呆。随后，一位路过的老奶奶走近，轻轻敲了敲魔术师的帽子，鸽子突然重新出现，而老奶奶也开始展现出令人惊叹的魔法技能，点燃了一场不可思议的魔法表演，改变了众人的一天。",
                "在一个宁静的村庄，居民们突然发现天空中出现了一架不明飞行物体，它发出闪烁的彩光并发出奇怪的声音。当村民们围拢过去观察时，他们发现这架飞行物体着陆后，一群友好的外星生物从中走出来，他们似乎是来地球进行文化交流的。这一不寻常的相遇引发了一场全球性的关注和探讨，人们开始探讨地球与外星生命的互动，以及未来的合作与发展。",
                "在一个宁静的海滨小镇，一场未曾有过的天文现象出现了，一颗流星雨异常明亮，照亮了整个夜空。人们走出户外，一同观赏这壮观景象，但随着流星雨消逝，他们发现陨石残骸开始落下，引发了短暂的恐慌。然而，当科学家前来研究时，他们惊讶地发现这些陨石碎片中蕴含着神秘的信息和技术，这一事件引发了全球性的科学研究和国际合作，以解开这些陨石的秘密。"
                ]
events_prompt = ''
for i in range(len(previous_evns)):
    event = previous_evns[i]
    events_prompt = events_prompt + f'事件: {event}'
summarize_prompt = '情概括下列的事件:' + events_prompt + '每个事件用一句话概括。'
summarized_prev_mem = get_response(summarize_prompt)
summarized_prev_mem = [eve for eve in summarized_prev_mem.split('\n')]
print(summarized_prev_mem)

['1. 研究员发现神秘地图，展开探险之旅。', '2. 山区村庄出现奇怪的光球，引发探究。', '3. 拆迁中发现古老地下城市，引发考古研究。', '4. 夜间海滩发现闪烁生物，引发科学研究和观赏活动。', '5. 乡村发现奇怪植物，引发医学和草药学研究。', '6. 考古队伍发现古老城市废墟和地下迷宫，引发历史研究和考古发展。', '7. 修道士实现心灵投影，引发精神和哲学变革。', '8. 小镇目击神秘灯光和不明飞行物，引发科学研究和探究。', '9. 湖泊中出现巨大水生生物，引发研究和探险。', '10. 街头魔术师和老奶奶展现出不可思议的魔法技能。', '11. 村庄出现不明飞行物体和外星生物，引发全球关注和探讨。', '12. 流星雨引发科学研究和国际合作，解开陨石的秘密。']


In [34]:
### 定义embedding获取函数
def get_embedding(text, model="text-embedding-ada-002"):
    openai_api_key = "548e5c0c2aff453e932948927a27bde6"
    openai.api_key = openai_api_key
    openai.api_type = "azure"
    # openai.api_version = "2023-06-15-preview"
    openai.api_version = "2023-03-15-preview"
    openai.api_base = "https://bigo-chatgpt-9.openai.azure.com/"
    res = openai.Embedding.create(
    input=[text], deployment_id="text-embedding-ada-002")['data'][0]['embedding']
    return res


In [18]:
emb1 = get_embedding(summarized_prev_mem[0])
emb2 = get_embedding(summarized_prev_mem[1])
print(emb1)

[0.0015974686248227954, -0.015618929639458656, -0.002127666724845767, -0.01398279145359993, -0.030632900074124336, 0.02437707968056202, -0.03621501848101616, 0.01480773463845253, -0.0004045230452902615, -0.01802501454949379, 0.009871823713183403, 0.03899232670664787, -0.0034424203913658857, -0.012607885524630547, -0.008696278557181358, -0.00019066076492890716, -0.0016206701984629035, 0.011232979595661163, 0.019743647426366806, 0.005173082929104567, -0.008538165129721165, 0.002821994014084339, 0.012195413932204247, -0.012099171057343483, -0.022960925474762917, -0.0009143123170360923, 0.0266044270247221, -0.02466581016778946, 0.012546014972031116, -0.0006513615953736007, 0.019028695300221443, 0.008015700615942478, -0.035005100071430206, -0.0214485302567482, -0.006369250826537609, 0.003791302675381303, 0.010655519552528858, -0.008194438181817532, 0.0007918598130345345, 0.013274715282022953, -0.0013070197310298681, 0.01864372193813324, -0.008263183757662773, -0.0027704350650310516, -0.0048

In [35]:
### 定义匹配函数，对于每个当下接收到的信息 perceived，将perceived每个事件的embedding与过往事件的embedding做向量匹配
### 匹配方式是余弦相似度，每个perceived事件返回最高的两个过往事件
def get_revelant_past_mem(perceived, prev_mem, top_k=2):
    sim_score = [[0 for _ in range(len(prev_mem))] for _ in range(len(perceived))]
    relevent_idx = {}
    perceived_emb = [get_embedding(perceived[i]) for i in range(len(perceived))]
    prev_mem_emb = [get_embedding(prev_mem[i]) for i in range(len(prev_mem))]
    for i in range(len(perceived)):
        for j in range(len(prev_mem)):
            sim_score[i][j] = cos_sim(perceived_emb[i], prev_mem_emb[j])
        relevent_idx[i], _ = get_top_k_idx(sim_score[i])
    return relevent_idx, perceived_emb, prev_mem_emb

def cos_sim(a, b):
  ## 返回一个值 0-1 
  return dot(a, b)/(norm(a)*norm(b))


def get_top_k_idx(scores, top_k=2):
    t = copy.deepcopy(scores)
    # 求m个最大的数值及其索引
    max_number = []
    max_index = []
    for _ in range(top_k):
        number = max(t)
        index = t.index(number)
        t[index] = 0
        max_number.append(number)
        max_index.append(index)
    return max_index, max_number

print(get_revelant_past_mem(perceived=perceived, prev_mem=summarized_prev_mem))

({0: [0, 7], 1: [8, 7], 2: [8, 11], 3: [4, 10], 4: [1, 9], 5: [8, 10], 6: [8, 3], 7: [5, 2], 8: [8, 11]}, [[-0.003990920726209879, 0.0008667151560075581, -0.0014613220700994134, -0.03826984018087387, -0.030959870666265488, 0.016823679208755493, -0.012140730395913124, -0.014297440648078918, -0.019887417554855347, -0.010790267959237099, 0.011253859847784042, 0.00934574194252491, -0.008411839604377747, 0.019591793417930603, 0.007786998990923166, -0.00810277834534645, 0.018678046762943268, -0.0033005722798407078, 0.0276811346411705, -0.006567550823092461, -0.014539314433932304, 0.01627274416387081, -0.017522426322102547, -0.016581805422902107, -0.005418649408966303, -0.006695206742733717, 0.026874888688325882, -0.003174596233293414, 0.01929616928100586, -0.018046487122774124, 0.02313927933573723, -0.002496005268767476, -0.03050299733877182, -0.006463410798460245, -0.017723988741636276, 0.011986199766397476, 0.010165426880121231, -0.0246442724019289, 0.0003821273276116699, -0.03259924054145

In [36]:
### 对于每一个当下接收到的信息，也就是perceived，在所有过往的事件中找2件与之最类似的
### 对于找到的事件，

reterived_mem_dic, _, _ = get_revelant_past_mem(perceived=perceived, prev_mem=summarized_prev_mem)
reterived_mem_idx = []
for k in reterived_mem_dic.keys():
    for i in range(len(reterived_mem_dic[k])):
        if reterived_mem_dic[k][i] not in reterived_mem_idx:
            reterived_mem_idx.append(reterived_mem_dic[k][i])
reterived_mem_idx

[0, 7, 8, 11, 4, 10, 1, 9, 3, 5, 2]

## Plan

In [37]:
### Plan 的输入是人物persona，迷宫整体maze，事件new_day，和被查找到的记忆reterived
### 如果是第一天或者是新的一天，所有plan将会被重新制定，否则则是执行之前已经规划好的计划
### 首先根据persona的记忆，我们知道会有recency，importance等去评判国王记忆的优先级，这时候会对过往记忆打一个优先级的标签
### 因为是简陋版所以这里直接假定给出优先级，选最高的前两个

focused_event = [summarized_prev_mem[reterived_mem_idx[i]] for i in range(len(reterived_mem_idx))]
focused_event 

['1. 研究员发现神秘地图，展开探险之旅。',
 '8. 小镇目击神秘灯光和不明飞行物，引发科学研究和探究。',
 '9. 湖泊中出现巨大水生生物，引发研究和探险。',
 '12. 流星雨引发科学研究和国际合作，解开陨石的秘密。',
 '5. 乡村发现奇怪植物，引发医学和草药学研究。',
 '11. 村庄出现不明飞行物体和外星生物，引发全球关注和探讨。',
 '2. 山区村庄出现奇怪的光球，引发探究。',
 '10. 街头魔术师和老奶奶展现出不可思议的魔法技能。',
 '4. 夜间海滩发现闪烁生物，引发科学研究和观赏活动。',
 '6. 考古队伍发现古老城市废墟和地下迷宫，引发历史研究和考古发展。',
 '3. 拆迁中发现古老地下城市，引发考古研究。']

In [39]:
### 接下来设定react model， react mode是根据当前接收到的信息和迷宫信息判断的

#reaction_mode = _should_react(persona, focused_event, personas)

### 比如如果检测到附近有人在旁边，react mode = chat，如果有床之类的可以交互的道具， react mode = play
### 如果是没有什么其他的，react mode = wait

plan_prompt = (
            "你是沙盒世界的一员，你的信息是{person_info}。\n"
            "你现在正在面对一些场景\n"
            "{scenarios}\n"
            "你遇到的近期经历是\n"
            "{asso_mem}\n"
            "你的类似过往经历是\n"
            "{focused_env}\n"
            "现在请根据这些经历，制定你的下一步计划\n"
        )
person_info = zhangsan.get_basic_info_prompt()
#print(person_info)

scenarios = ''
for i in range(len(perceived)):
    event = perceived[i]
    scenarios = scenarios + f'{event}\n'
#print(scenarios)

asso_mem = ''
for i in range(len(summarized_prev_mem)):
    event = summarized_prev_mem[i]
    asso_mem = asso_mem + f'{event}\n'
#print(asso_mem)

focused_env = ''
for i in range(len(focused_env)):
    event = focused_env[i]
    asso_mem = asso_mem + f'{event}\n'
#rint(focused_env)

plan_pref = plan_prompt.format_map({'person_info':person_info, 'scenarios':scenarios, 'asso_mem':asso_mem, 'focused_env':focused_env})

plan = get_response(plan_pref)
print(plan)

我的下一步计划是选择一项经历并深入研究，探究其中的奥秘和可能的影响。我将利用我的好奇心和热情，积极参与研究和探究活动，以期获得更多知识和体验。同时，我也将继续发掘自己的技能和兴趣爱好，不断提升自己的能力和素质。最终，我希望能够为世界的发展和进步做出自己的贡献。


## Reflect

In [40]:
### 根据做出的计划，做出一定的总结和思考

reflect_prompt = ("根据以下信息：\n\n\n"
                  "{plan_pref}"
                  "你制定了计划：\n\n\n"
                  "{plan}"
                  "请根据你获得的信息和你的计划做出总结和思考，以第一人称的口吻，只返回反思内容：\n\n")

reflect_pref = reflect_prompt.format_map({'plan_pref':plan_pref, 'plan':plan})
reflect = get_response(reflect_pref)
print(reflect)
zhangsan.asso_mem.append(reflect)

我意识到自己是一个充满好奇心和热情的人，乐于探索和发现新事物。在面对各种经历时，我可以运用自己的技能和性格特点，积极参与研究和探究活动，并在其中获得更多知识和体验。我深信，通过不断提升自己的能力和素质，我可以为世界的发展和进步做出自己的贡献。
